# UNIVARIATE NONLINEAR REGRESSION

In [64]:
import numpy
import pandas
import tensorflow
from tensorflow import keras
from matplotlib import pyplot
import seaborn
import itertools
import datetime
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, r2_score
from tensorflow.python.client import device_lib

%matplotlib inline

In [65]:
tensorflow.config.list_physical_devices('GPU')

[]

In [66]:
# DEBUGGING ANNOYING WARNING
# tensorflow.debugging.set_log_device_placement(True)
from tensorflow.python.data.util import options as options_lib
from tensorflow.python.data.experimental import DistributeOptions, AutoShardPolicy

DistributeOptions.auto_shard_policy = options_lib.create_option(
    name="auto_shard_policy",
    ty=AutoShardPolicy,
    docstring="The type of sharding to use. See "
    "`tf.data.experimental.AutoShardPolicy` for additional information.",
    default_factory=lambda: AutoShardPolicy.DATA,
)

## INPUT PARAMETERS HERE

In [67]:
# DATA PRE-PREPARATION
file_path = '../../../dataset/occurrences.xlsx'

df = pandas.read_excel(file_path)
df = df.drop(["rai", "obm_afeto", "qualificacao", "sexo"], axis=1).loc[0:200,:] 

#REMOVING NULL VALUES
df.loc[pandas.isnull(df["data"])]
df.loc[pandas.isnull(df["naturezas"])]
df.loc[pandas.isnull(df["bairro_cidade"])]
df.loc[pandas.isnull(df["tr"])]
df.loc[pandas.isnull(df["obm_escala"])]

df = df.loc[df["bairro_cidade"] != "(null)"]
df = df.loc[df["recurso"] != "(null)"]
df = df.loc[df["tr"] != "(null)"]
df = df.loc[df["obm_escala"] != "(null)"]

#TRANSFORMING "data" COLUMN INTO NEW COLUMNS "dia" e "periodo". ALSO TRANSFORMING "tr" COLUMN INTO "tempo_resposta" COLUMN

def day_name(timestamp):
    weekdays = ('Segunda-feira', 'Terça-feira', 'Quarta-feira', 'Quinta-feira', 'Sexta-feira', 'Sábado', 'Domingo')
    return weekdays[timestamp.weekday()]

def period_of_day(timestamp):
    period = ("Madrugada", "Matutino", "Vespertino", "Noturno")
    # Madrugada 00:00 às 05:59
    # Matutino 06:00 às 11:59
    # Vespertino 12:00 às 17:59
    # Noturno 18:00 às 23:59
    if timestamp.hour >= 0 and timestamp.hour < 6:
        return period[0]
    elif timestamp.hour >= 6 and timestamp.hour < 12:
        return period[1]
    elif timestamp.hour >= 12 and timestamp.hour < 18:
        return period[2]
    elif timestamp.hour >= 18 and timestamp.hour < 24:
        return period[3]



def response_time(response_time):
    # Muito rápido 0 a 10 minutos
    # Rápido 10 a 15 minutos
    # Médio 15 a 20 minutos
    # Longo 20 a 30 minutos
    # Muito longo 30 a 45 minutos
    # Extremamente longo > 45 minutos

    response_time_metric = ("Muito rápido", "Rápido", "Médio", "Longo", "Muito longo", "Extremamente longo")

    if type(response_time) is datetime.time:

        total_time_in_minutes = response_time.hour*60 + response_time.minute + response_time.second/60

        if total_time_in_minutes >= 0 and total_time_in_minutes <= 10:
            return response_time_metric[0]
        elif total_time_in_minutes > 10 and total_time_in_minutes <= 15:
            return response_time_metric[1]
        elif total_time_in_minutes > 15 and total_time_in_minutes <= 20:
            return response_time_metric[2]
        elif total_time_in_minutes > 20 and total_time_in_minutes <= 30:
            return response_time_metric[3]
        elif total_time_in_minutes > 30 and total_time_in_minutes <= 45:
            return response_time_metric[4]
        elif total_time_in_minutes > 45:
            return response_time_metric[5]
        
#Lembrar de remover os valores que não são datetime.time do df["tr"]
df.loc[:, "dia"] = df["data"].apply(day_name)
df.loc[:, "periodo"] = df["data"].apply(period_of_day)
df.loc[:, "tempo_resposta"] = df["tr"].apply(response_time)

#REMOVING "DATA" AND "TR" COLUMNS

df = df.drop(["data", "tr"], axis=1)

#Removing None values
df = df.dropna()
df = df.mask(df.eq('None')).dropna()
df = df.astype(str)
df

/home/higoreller/miniconda3/envs/condaenv/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Cell F221384 is marked as a date but the serial value -736954.9663425926 is outside the limits for dates. The cell will be treated as an error.
  for idx, row in parser.parse():


,naturezas,bairro_cidade,recurso,obm_escala,dia,periodo,tempo_resposta
1,RESGATE (3)->EMERGÊNCIA CLÍNICA (304)->CRISE H...,VILA OLIVEIRA - APARECIDA DE GOIÂNIA,UR-226,7º BBM - APARECIDA DE GOIÂNIA,Domingo,Madrugada,Rápido
2,BUSCA E SALVAMENTO (4)->PESSOAS (403)->RETIRAD...,LUCILENE - SANTA HELENA DE GOIÁS,UR-171,3ª CIBM - SANTA HELENA,Domingo,Madrugada,Rápido
3,RESGATE (3)->ACIDENTE DE TRÂNSITO (301)->CARRO...,SÍTIOS SANTA LUZIA - APARECIDA DE GOIÂNIA,UR-140,7º BBM - APARECIDA DE GOIÂNIA,Domingo,Madrugada,Longo
4,INCÊNDIO URBANO (6)->VEÍCULOS (610)->VEÍCULO D...,ZONA RURAL - NIQUELÂNDIA,ABTS-07,6ª CIBM - NIQUELÂNDIA,Domingo,Madrugada,Muito rápido
5,RESGATE (3)->EMERGÊNCIA CLÍNICA (304)->ACIDENT...,ZONA RURAL - CATALÃO,UR-181,10º BBM - CATALÃO,Domingo,Madrugada,Muito rápido
...,...,...,...,...,...,...,...
196,RESGATE (3)->ACIDENTE DE TRÂNSITO (301)->CAPOT...,ZONA RURAL - ITAUÇU,ASA-77,7ª CIBM - INHUMAS,Domingo,Vespertino,Extremamente longo
197,RESGATE (3)->ACIDENTE DE TRÂNSITO (301)->CAPOT...,ZONA RURAL - ITAUÇU,UR-189,7ª CIBM - INHUMAS,Domingo,Vespertino,Extremamente longo
198,RESGATE (3)->ACIDENTE DE TRÂNSITO (301)->CAPOT...,ZONA RURAL - ITAUÇU,UR-189,7ª CIBM - INHUMAS,Domingo,Vespertino,Extremamente longo
199,RESGATE (3)->ACIDENTE DE TRÂNSITO (301)->CAPOT...,ZONA RURAL - ITAUÇU,UR-200,7ª CIBM - INHUMAS,Domingo,Vespertino,Extremamente longo


In [68]:
# DATA PARAMETERS

var_types = {'naturezas' : 'str',
             'bairro_cidade' : 'str',
             'recurso' : 'str',
             'dia': 'str',
             'periodo': 'str',
             'tempo_resposta': 'str',
             'obm_escala': 'str',
             }

# COOK PARAMETERS
reg_col = 'tempo_resposta'
folding_sampler = 'tempo_resposta'
my_metric = 'mse'
relevant_features = 4
my_patience = 50
min_improvement = 0.01
min_neuron_gain = 0.025
random_seed = 42
n_k_folds = 5
learning_rate = 0.01
hidden_activations = ['linear', 'sigmoid']
print_anyway = True
rollback_on_no_bound_gain = True
my_optimizers = ['adam']

In [69]:
# IMPORT AND CONVERT
for input_col in var_types.keys():
    df[input_col] = df[input_col].astype(var_types[input_col])

# SCATTER PLOT
#g = seaborn.pairplot(df, hue='naturezas', height=3, diag_kind='kde')
#_ = g.map_lower(seaborn.kdeplot, levels=1, color=".2")

## ROBUST COOKING

In [70]:
# SET THE RANDOM SEED
numpy.random.seed(random_seed)
tensorflow.random.set_seed(random_seed)

# QUANTIFY REPRESENTATION 
dfn_mi_list = []
dfn_data_list = []
for my_var in var_types.keys():
    if my_var == reg_col:
        col_class = 'Y'
    else:
        col_class = 'X'

    if var_types[my_var] == 'float': # NUMERIC DATA
        dfn_mi_list.append((col_class, 'continuous', my_var))
        dfn_data_list.append(df[[my_var]].values)

    elif var_types[my_var] == 'str': # CATEGORICAL DATA
        one_hot = OneHotEncoder(sparse=False)
        var_cat = df[[my_var]].to_numpy()
        one_hot.fit(var_cat)
        cat_df = pandas.DataFrame(one_hot.transform(var_cat))
        cat_df.columns = one_hot.categories_
        for cat in cat_df.columns:
            dfn_mi_list.append((col_class, 'categorical', cat[0]))
            dfn_data_list.append(cat_df[[cat]].values)

dfn_mi = pandas.MultiIndex.from_tuples(dfn_mi_list)
dfn = pandas.DataFrame(index=df.index, columns=dfn_mi, data=numpy.concatenate(dfn_data_list, axis=1))
dfn = dfn.sort_index(axis=1)
dfn

X                                                          \
         categorical                                                           
    10ª CIBM - POSSE 10º BBM - CATALÃO 11ª CIBM - URUAÇU 11º BBM - PORANGATU   
1                0.0               0.0               0.0                 0.0   
2                0.0               0.0               0.0                 0.0   
3                0.0               0.0               0.0                 0.0   
4                0.0               0.0               0.0                 0.0   
5                0.0               1.0               0.0                 0.0   
..               ...               ...               ...                 ...   
196              0.0               0.0               0.0                 0.0   
197              0.0               0.0               0.0                 0.0   
198              0.0               0.0               0.0                 0.0   
199              0.0               0.0               0.0                 0.0   
200              0.0               0.0               0.0                 0.0   

                                                          \
                                                           
    12ª CIBM - MORRINHOS 12º BBM - GOIÁS 13º BBM - JATAÍ   
1                    0.0             0.0             0.0   
2                    0.0             0.0             0.0   
3                    0.0             0.0             0.0   
4                    0.0             0.0             0.0   
5                    0.0             0.0             0.0   
..                   ...             ...             ...   
196                  0.0             0.0             0.0   
197                  0.0             0.0             0.0   
198                  0.0             0.0             0.0   
199                  0.0             0.0             0.0   
200                  0.0             0.0             0.0   

                                                                              \
                                                                               
    14ª CIBM - PIRES DO RIO 14º BBM - SENADOR CANEDO 15ª CIBM - QUIRINÓPOLIS   
1                       0.0                      0.0                     0.0   
2                       0.0                      0.0                     0.0   
3                       0.0                      0.0                     0.0   
4                       0.0                      0.0                     0.0   
5                       0.0                      0.0                     0.0   
..                      ...                      ...                     ...   
196                     0.0                      0.0                     0.0   
197                     0.0                      0.0                     0.0   
198                     0.0                      0.0                     0.0   
199                     0.0                      0.0                     0.0   
200                     0.0                      0.0                     0.0   

     ...  \
     ...   
     ...   
1    ...   
2    ...   
3    ...   
4    ...   
5    ...   
..   ...   
196  ...   
197  ...   
198  ...   
199  ...   
200  ...   

                                                                            Y  \
                                                                  categorical   
    RESGATE (3)->EMERGÊNCIA CLÍNICA (304)->OUTRO / EMERGÊNCIA CLÍNICA (30499)   
1                                                  0.0                          
2                                                  0.0                          
3                                                  0.0                          
4                                                  0.0                          
5                                                  0.0                          
..                                                 ...                          
196                                   

In [71]:
# NUMPY MATRIX FOR TRAINING
X = dfn['X'].to_numpy()
my_vars = list(dfn['X'].columns.get_level_values(1))
Y = dfn['Y'].to_numpy()

# NORMALIZE VARIABLES
X_scaler = StandardScaler()
X_scaler = X_scaler.fit(X)
X_scaled = X_scaler.transform(X)

# TRANSFORM Y
Y_scaler = None
if var_types[reg_col] == 'float': # IF REGRESSION NORMALIZE
    Y_scaler = StandardScaler()
    Y_scaler = Y_scaler.fit(Y)
    Y_scaled = Y_scaler.transform(Y)
else:
    Y_scaled = Y

# Y SAMPLING
Y_sample = df[[folding_sampler]].to_numpy()

# GET STRATIFIED FOLDS
sk_folds = StratifiedKFold(n_splits=n_k_folds)
sk_folds.get_n_splits(X_scaled, Y_sample)

# TRAINING LOOP
n_classes = dfn['Y'].columns.shape[0]
hidden_layers = 1

# PATTERN SEARCH
search_vector = numpy.identity(len(hidden_activations), dtype=int)
current_configurations = search_vector

still_searching = True
best_net_name = None
best_net = None
best_config = None

if my_metric == 'mse':
    my_mode = 'min'
    best_metric = 1.0
    best_metric_val = 1.0
    metric_bound = 1000.0
else:
    my_mode = 'max'
    best_metric = 0.0
    best_metric_val = 0.0
    metric_bound = 0.0


result_list = []

# RUNNING ON GPU AND CPU
strategy = tensorflow.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [72]:
print('COOKING BEST NETWORK STARTED...\n')
while still_searching:
    for net_config in current_configurations:
        print('CONFIG: {}'.format(net_config))
        ix_fold = 1
        for train_index, test_index in sk_folds.split(X_scaled, Y_sample):
            print('FOLD: {}/{}'.format(ix_fold, n_k_folds))

            # GET STRATIFIED DATA
            X_train, X_test = X_scaled[train_index], X_scaled[test_index]
            Y_train, Y_test = Y_scaled[train_index], Y_scaled[test_index]
            for optimizer in my_optimizers:

                # Open a strategy scope and create/restore the model
                with strategy.scope():
                    # DECLARE OPTIMIZERS INSIDE SCOPE
                    _optimizers = {'adagrad': tensorflow.keras.optimizers.Adagrad(learning_rate=learning_rate),
                                   'rmsprop': tensorflow.keras.optimizers.RMSprop(learning_rate=learning_rate),
                                   'adam': tensorflow.keras.optimizers.Adam(learning_rate=learning_rate)}
                    # DECLARE NET
                    inputs = keras.Input(shape=(X_train.shape[1], ))
                    hidden_layer_list = []
                    net_name = ''
                    for ix_activation in numpy.arange(len(net_config)):
                        f_activation = hidden_activations[ix_activation]
                        n_neuron = net_config[ix_activation]
                        if n_neuron > 0:
                            temp_hidden_layer = keras.layers.Dense(n_neuron, activation=f_activation)(inputs)
                            hidden_layer_list.append(temp_hidden_layer)
                            net_name += '{}{}'.format(f_activation, n_neuron)
                    if len(hidden_layer_list) > 1:
                        hidden = keras.layers.Concatenate(axis=1)(hidden_layer_list)
                    else:
                        hidden = hidden_layer_list[0]
                    outputs = keras.layers.Dense(n_classes, activation='linear')(hidden)
                    model_name = '{}_{}_F{}_WRAPPER'.format(net_name.upper(), optimizer.upper(), ix_fold)
                    my_net = keras.Model(inputs=inputs, outputs=outputs, name=model_name)

                # COMPILE NET
                metric_name = my_metric.lower()
                val_metric_name = 'val_{}'.format(metric_name)
                my_net.compile(optimizer=_optimizers[optimizer], loss='mse', metrics=[my_metric])
                es_loss = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', patience=my_patience, restore_best_weights=True)
                es_metric = tensorflow.keras.callbacks.EarlyStopping(monitor=metric_name, mode=my_mode, patience=my_patience, min_delta=min_improvement)
                es_val_metric = tensorflow.keras.callbacks.EarlyStopping(monitor=val_metric_name, mode=my_mode,
                                                                         patience=n_k_folds*my_patience, min_delta=min_improvement)

                # FIT WRAPPER
                history = my_net.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=X_train.shape[0],
                                    epochs=10*my_patience, verbose=0, callbacks=[es_loss, es_metric, es_val_metric])

                # CALCULATE VARIABLE RELEVANCE
                Y_est = my_net.predict(X_scaled, verbose=0)
                relevance_dict = {}
                for feature in my_vars:
                    X_relevance = numpy.copy(X_scaled)
                    X_relevance[:, my_vars == feature] = 0.0
                    Y_est_relevance = my_net.predict(X_relevance, verbose=0)
                    Y_relevance = (Y_est - Y_est_relevance)**2
                    feature_relevance = Y_relevance.sum()/Y_relevance.shape[0]
                    relevance_dict[feature] = feature_relevance
                s_relevance = pandas.Series(relevance_dict)
                s_relevance_norm = s_relevance/s_relevance.max()
                s_relevance_norm = s_relevance_norm.sort_values(ascending=False).head(relevant_features)
                selected_features = list(s_relevance_norm.index.values)
                selected_relevance = list(100.0*s_relevance_norm.values)

                # FINAL NET
                with strategy.scope():
                    inputs = keras.Input(shape=(len(selected_features), ))
                    hidden_layer_list = []
                    net_name = ''
                    for ix_activation in numpy.arange(len(net_config)):
                        f_activation = hidden_activations[ix_activation]
                        n_neuron = net_config[ix_activation]
                        if n_neuron > 0:
                            temp_hidden_layer = keras.layers.Dense(n_neuron, activation=f_activation)(inputs)
                            hidden_layer_list.append(temp_hidden_layer)
                            net_name += '{}{}'.format(f_activation, n_neuron)
                    if len(hidden_layer_list) > 1:
                        hidden = keras.layers.Concatenate(axis=1)(hidden_layer_list)
                    else:
                        hidden = hidden_layer_list[0]
                    outputs = keras.layers.Dense(n_classes, activation='linear')(hidden)
                    model_name = '{}_{}_F{}'.format(net_name.upper(), optimizer.upper(), ix_fold)
                    my_net = keras.Model(inputs=inputs, outputs=outputs, name=model_name)

                # COMPILE NET
                metric_name = my_metric.lower()
                val_metric_name = 'val_{}'.format(metric_name)
                my_net.compile(optimizer=_optimizers[optimizer], loss='mse', metrics=[my_metric])
                es_loss = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', patience=my_patience, restore_best_weights=True)
                es_metric = tensorflow.keras.callbacks.EarlyStopping(monitor=metric_name, mode=my_mode, patience=my_patience, min_delta=min_improvement)
                es_val_metric = tensorflow.keras.callbacks.EarlyStopping(monitor=val_metric_name, mode=my_mode,
                                                                        patience=n_k_folds*my_patience, min_delta=min_improvement)

                # FIT WRAPPER
                ix_relevant = dfn['X'].columns.get_level_values(1).isin(selected_features)
                history = my_net.fit(X_train[:, ix_relevant], Y_train, validation_data=(X_test[:, ix_relevant], Y_test), batch_size=X_train.shape[0],
                                    epochs=10*my_patience, verbose=0, callbacks=[es_loss, es_metric, es_val_metric])

                # UNPACK TRAINING RESULTS
                model_loss = history.history['loss'][-1]
                model_val_loss = history.history['val_loss'][-1]
                metric_val = history.history[metric_name][-1]
                val_metric_val = history.history[val_metric_name][-1]
                n_epochs = len(history.history['val_loss'])

                # CHECK IF BEST NET
                best_net_txt = ''
                is_best = False
                if my_mode == 'max':
                    is_best = ((metric_val > best_metric + min_improvement) & (val_metric_val >= best_metric_val)) | ((metric_val >= best_metric) & (val_metric_val > best_metric_val + min_improvement))
                elif my_mode == 'min':
                    is_best = ((metric_val < best_metric - min_improvement) & (val_metric_val <= best_metric_val)) | ((metric_val <= best_metric) & (val_metric_val < best_metric_val - min_improvement))

                if is_best:
                    best_net_name = model_name
                    best_optimizer = optimizer
                    best_config = net_config
                    best_features = selected_features
                    best_relevance = selected_relevance
                    best_net = my_net
                    best_neurons = net_config.sum()
                    best_metric = metric_val
                    best_metric_val = val_metric_val
                    best_net_txt = '**Best NET!**'

                result_dict = {'model_name': model_name, 'optimizer': optimizer, 'net_config': net_config, 'neurons': net_config.sum(), 'fold': ix_fold, 'features': selected_features, 'relevance': selected_relevance, 'loss': model_loss,
                    'val_loss': model_val_loss, metric_name: metric_val, val_metric_name: val_metric_val, 'epochs': n_epochs, 'is_best': is_best, 'net': my_net}
                result_list.append(result_dict)
                if is_best or print_anyway:
                    print('{}_{} > epochs: {} loss: {:.2f} val_loss: {:.2f} {}: {:.2f} {}: {:.2f} {}'.format(model_name,'_'.join(selected_features), n_epochs, model_loss, model_val_loss, metric_name, metric_val, val_metric_name, val_metric_val, best_net_txt))

                # IF MAXIMIZED METRICS BREAK
                if my_mode == 'max':
                    maximized_metrics = (val_metric_val == 1.0) & (metric_val == 1.0)
                elif my_mode == 'min':
                    maximized_metrics = (val_metric_val == 0.0) & (metric_val == 0.0)

                if maximized_metrics:
                    break
            
            if maximized_metrics:
                break

            ix_fold += 1

        if maximized_metrics:
            break

    # TEST IF MORE NEURONS ARE NECESSARY
    dont_need_extra_neurons = False
    total_neurons = net_config.sum()

    if total_neurons > 1:
        
        # INCREASING NEURONS DIDNT ACHIEVE BEST NET
        if best_neurons < total_neurons:
            dont_need_extra_neurons = True
            print('NETs w {} neurons didnt improve {}!'.format(total_neurons, metric_name))
            break
    
    # CALCULATE CONFIG BOUND
    df_results = pandas.DataFrame(result_list)
    ix_config = numpy.array([numpy.linalg.norm(best_config - x) for x in df_results.net_config]) == 0.0
    if my_mode == 'max':
        bound_est = df_results[(df_results.optimizer == best_optimizer) & ix_config][val_metric_name].min()
    elif my_mode == 'min':
        bound_est = df_results[(df_results.optimizer == best_optimizer) & ix_config][val_metric_name].max()

    # TEST IF LOWER BOUND IMPROVED
    bound_didnt_improve = False
    if total_neurons > 1:
        neuron_gain = 0.0
        if my_mode == 'max':
            if bound_est > metric_bound:
                if metric_bound > 0.0:
                    neuron_gain = bound_est/metric_bound - 1.0
                    if neuron_gain <= min_neuron_gain:
                        print('NETs w {} neurons didnt improve {} {:.2f} neuron gain {:.2f} / min {:.2f}!'.format(total_neurons, val_metric_name, bound_est, 100.0*neuron_gain, 100.0*min_neuron_gain))
                        bound_didnt_improve = True
                        break
                    else:
                        print('NETs Neuron Gain w/ {} neurons: {:.2f} >> {:.2f} = {:.2f}%'.format(total_neurons, metric_bound, bound_est ,100.0*neuron_gain))
                else:
                    print('Best Config Bound Estimation{:.2f}'.format(bound_est))
            else:
                print('No Bound Gain on {} neurons for {}: {:.2f} >> {:.2f}'.format(total_neurons, best_optimizer, metric_bound, bound_est))
                bound_didnt_improve = True
                break
        elif my_mode == 'min':
            if bound_est < metric_bound:
                if metric_bound > 0.0:
                    neuron_gain = 1.0 - bound_est/metric_bound
                    if neuron_gain <= min_neuron_gain:
                        print('NETs w {} neurons didnt improve {} {:.2f} neuron gain {:.2f} / min {:.2f}!'.format(total_neurons, val_metric_name, bound_est, 100.0*neuron_gain, 100.0*min_neuron_gain))
                        bound_didnt_improve = True
                        break
                    else:
                        print('NETs Neuron Gain w/ {} neurons: {:.2f} >> {:.2f} = {:.2f}%'.format(total_neurons, metric_bound, bound_est ,100.0*neuron_gain))
                else:
                    print('Best Config Bound Estimation{:.2f}'.format(bound_est))
            else:
                print('No Bound Gain on {} neurons for {}: {:.2f} >> {:.2f}'.format(total_neurons, best_optimizer, metric_bound, bound_est))
                bound_didnt_improve = True
                break

    else:
        print('BEST CONFIG {} & best quality for {}: {:.2f}'.format(best_config, best_optimizer.upper(), bound_est))
    
    metric_bound = bound_est

    # ALTER CONFIG FROM BEST CONFIG IF NEEDED
    if maximized_metrics or dont_need_extra_neurons or bound_didnt_improve:
        still_searching = False
    else:
        current_configurations = best_config + search_vector

COOKING BEST NETWORK STARTED...

CONFIG: [1 0]
FOLD: 1/5


/home/higoreller/miniconda3/envs/condaenv/lib/python3.9/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
2022-08-17 15:16:21.465254: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-08-17 15:16:51.472606: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


LINEAR1_ADAM_F1_10ª CIBM - POSSE_10º BBM - CATALÃO_11ª CIBM - URUAÇU_11º BBM - PORANGATU > epochs: 75 loss: 0.96 val_loss: 1.07 mse: 0.96 val_mse: 1.07 
FOLD: 2/5


2022-08-17 15:17:23.420754: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-08-17 15:17:53.471703: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


LINEAR1_ADAM_F2_10ª CIBM - POSSE_10º BBM - CATALÃO_11ª CIBM - URUAÇU_11º BBM - PORANGATU > epochs: 71 loss: 0.96 val_loss: 1.10 mse: 0.96 val_mse: 1.10 
FOLD: 3/5


...

LINEAR1SIGMOID1_ADAM_F3_10ª CIBM - POSSE_10º BBM - CATALÃO_11ª CIBM - URUAÇU_11º BBM - PORANGATU > epochs: 87 loss: 0.95 val_loss: 1.09 mse: 0.95 val_mse: 1.09 
FOLD: 4/5


2022-08-17 15:36:45.742800: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-08-17 15:37:15.745827: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-08-17 15:37:46.450752: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


LINEAR1SIGMOID1_ADAM_F4_10ª CIBM - POSSE_10º BBM - CATALÃO_11ª CIBM - URUAÇU_11º BBM - PORANGATU > epochs: 74 loss: 1.00 val_loss: 0.91 mse: 1.00 val_mse: 0.91 
FOLD: 5/5


2022-08-17 15:38:16.501169: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-08-17 15:38:46.518129: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-08-17 15:39:17.583821: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


LINEAR1SIGMOID1_ADAM_F5_10ª CIBM - POSSE_10º BBM - CATALÃO_11ª CIBM - URUAÇU_11º BBM - PORANGATU > epochs: 78 loss: 0.99 val_loss: 0.95 mse: 0.99 val_mse: 0.95 
NETs w 2 neurons didnt improve mse!


In [73]:
# ROLL BACK TO LESS NEURON IF DIDNT IMPROVE
if bound_didnt_improve and rollback_on_no_bound_gain:
    best_row = df_results[(df_results.neurons == best_config.sum() - 1) & df_results.is_best].tail(1)
    best_net_name = best_row['model_name'].values[0]
    best_optimizer = best_row['optimizer'].values[0]
    best_net = best_row['net'].values[0]
    best_config = best_row['net_config'].values[0]
    best_features = best_row['features'].values[0]
    best_neurons = best_row['neurons'].values[0]
    best_metric = best_row[metric_name].values[0]
    best_metric_val = best_row[val_metric_name].values[0]

In [74]:
# PREDICT
ix_best_features = dfn['X'].columns.isin(best_features, level=1)
Y_est_scaled = best_net.predict(X_scaled[:, ix_best_features], verbose=0)
Y_est = Y_scaler.inverse_transform(Y_est_scaled)

# PLOT Y Vs YEST
df_result = pandas.DataFrame(index=df.index)
df_result['Y'] = Y
df_result['Y_est'] = Y_est
df_result['r'] = df_result['Y_est'] - df_result['Y']
df_result['rabs'] = df_result['r'].abs()
df_result = df_result.join(df.loc[:, df.columns != reg_col])

df_result

ValueError: Expected a 1D array, got an array with shape (169, 47)

In [ ]:
# STABLE RESIDUAL PLOT
r_mean = df_result.r.median()
r_std = df_result.r.std()
r2 = r2_score(df_result.Y.to_numpy(), df_result.Y_est.to_numpy())

fig = pyplot.figure(figsize=(16, 8))
ax = fig.add_subplot(121)
lines = seaborn.scatterplot(data=df_result, x='Y', y='Y_est', size='rabs', ax=ax, label='residuo', color='green')
_ = ax.plot([df_result.Y.min(), df_result.Y.max()], [df_result.Y.min(), df_result.Y.max()], color='green', label='ref')
_ = ax.grid(alpha=0.2)
_ = ax.set_title('{} R2: {:.1f}%'.format(best_net_name,100.0*r2), size=20)
ax2 = ax.twinx()
_ = seaborn.kdeplot(data=df_result, x='Y', ax=ax2, label='Y', color='green', fill=True, alpha=0.1, linewidth=0.7)
_ = seaborn.kdeplot(data=df_result, x='Y_est', ax=ax2, label='Y_est', color='green', fill=True, alpha=0.5, linewidth=0.7)
_ = ax2.set_ylabel('')
_ = ax2.set_yticks([])
_ = ax.legend(loc='upper right')
_ = ax2.legend(loc='lower right')
ax3 = fig.add_subplot(122)
_ = seaborn.scatterplot(data=df_result, x='r', y='Y', ax=ax3, label='residuo', color='red')
ax4 = ax3.twinx()
_ = seaborn.kdeplot(data=df_result, x='r', ax=ax4, label='residuo', color='red', fill=True, alpha=0.5, linewidth=0.7)
_ = ax4.grid(alpha=0.2)
_ = ax4.set_ylabel('')
_ = ax4.set_yticks([])
_ = ax4.set_title('Residuo: {:.2f} +- {:.2f}'.format(r_mean, r_std), size=20)
_ = ax3.plot([r_mean, r_mean], [df_result.Y_est.min(), df_result.Y_est.max()], '--', color='red')

: 

In [ ]:
df_result.Y_est.max()

: 

In [ ]:
# SCATTER PLOT
resplot_var = list(df.columns[df.columns != reg_col])
resplot_var.append('r')
g2 = seaborn.pairplot(df_result[resplot_var], hue='naturezas', height=3, diag_kind='kde')
# _ = g2.map_lower(seaborn.kdeplot, levels=1, color=".2")

: 

: 